<a href="https://colab.research.google.com/github/Existanze54/sirius-machine-learning-2025/blob/main/Seminars/BioInf/S5_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оновные алгоритмы ML

### Семинар 5. Методы кросс-валидации (CV). StratifiedShuffleSplit

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

# Использование встроенных средств sklearn

Для начала сгенерируем искусственный набор данных, как в прошлом семинаре, просто запустим следующую ячейку

In [ ]:
X, Y = datasets.make_classification(n_features=2,
                                       n_redundant=0,
                                       n_informative=1,
                                       random_state=202,
                                       n_clusters_per_class=1,
                                       flip_y=0,
                                       n_samples=200)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
?StratifiedShuffleSplit.split


Скопируйте и поменяйте код из лекции для Monte-Carlo cross-validation так, чтобы он использовал <a href='https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html'>StratifiedShuffleSplit</a>

In [ ]:
def get_monte_carlo_cv_scores(X, y, clf, n_samples,
                              test_size='random', random_state=12345):
    rng = np.random.RandomState(seed=random_state)
    seeds = np.arange(10**5)
    rng.shuffle(seeds)
    seeds = seeds[:n_samples]
    auroc_mccv = []
    if test_size == 'random':
        sizes = rng.choice(np.linspace(0.05, 0.5, 46),
                           size=n_samples, replace=True)
        for i, size in zip(seeds, sizes):
            X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                              test_size=size,
                                                              random_state=i,
                                                              stratify=y)
            scaler = StandardScaler().fit(X_train)
            X_train = scaler.transform(X_train)
            X_val = scaler.transform(X_val)
            y_score = clf.fit(X_train, y_train).predict_proba(X_val)[:, 1]
            auroc = roc_auc_score(y_true=y_val, y_score=y_score)
            auroc_mccv.append(auroc)
    elif isinstance(test_size, float):
        for i in seeds:
            X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                              test_size=test_size,
                                                              random_state=i,
                                                              stratify=y)
            scaler = StandardScaler().fit(X_train)
            X_train = scaler.transform(X_train)
            X_val = scaler.transform(X_val)
            y_score = clf.fit(X_train, y_train).predict_proba(X_val)[:, 1]
            auroc = roc_auc_score(y_true=y_val, y_score=y_score)
            auroc_mccv.append(auroc)
    else:
        return 'Wrong test_size type. Must be string ("random") or float'
    return np.array(auroc_mccv)

Если функция написана правильно, должны получиться похожие (или такие же?) графики, как в лекции

In [ ]:
# your code from lection here

# Предсказание растворимости веществ

Загрузим датасет [AqSolDB](https://www.nature.com/articles/s41597-019-0151-1), объединяющий информацию о растворимости химических веществ из разных источников.



In [ ]:
!wget  https://kodomo.fbb.msu.ru/FBB/year_20/ml/curated-solubility-dataset.csv

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv("curated-solubility-dataset.csv")

In [ ]:
data.head()

В качестве Y используйте колонку **Solubility**

In [ ]:
y = data['Solubility']
y.head()

В качестве X - все колонки после Solubility

In [ ]:
X = data.iloc[:, 6:]
X.head()

In [ ]:
X.Group.value_counts()

Разбейте ваши данные на обучение и тест в отношении 8:2

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=777)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

Попробуйте на данной задаче и на основе только посчитанных авторами признаков (начинаются с MolWt), предсказать растворимость веществ при помощи:

1. Линейной регрессии

2. KNN-регрессии

3. SVM (понадобится SVR из пакета)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn import svm
from sklearn.svm import SVR

Так как у второго и третьего алгоритма есть гиперпараметры, то сначала на трейне используйте GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split

In [ ]:
# пример из мануала для SVC- адаптируйте к вашей задаче
parameters = {'kernel':('linear', 'rbf'), 'C':[0.01, 0.1, 1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(X_train, y_train)

Для SVR используйте те же параметры, что указаны в ячейке.
Для KNN - число соседей

Выберите лучший на кросс-валидации алгоритм

Проверьте финальную модель на тесте

Какие проблемы вы видите в случайном разбиении, примененом нами

# "Парадокс"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score

Создадим таблицу 200 на 10000 со "здоровыми" и "больными" пациентами и данными об "экспрессии" их генов.


In [ ]:
gene_count = 10000
genes = [f"Gene{ind}" for ind in range(gene_count)]
healthy = pd.DataFrame(np.random.normal(size=(100, gene_count),
                                        loc=0,
                                        scale=1),
                       columns=genes)
healthy['State'] = "H"
diseased = pd.DataFrame(np.random.normal(size=(100, gene_count),
                                         loc=0,
                                         scale=1),
                        columns=genes)
diseased['State'] = "D"
patients = pd.concat([healthy, diseased], axis=0)

In [ ]:
patients

In [ ]:
X = patients.drop("State", axis=1)
Y = patients['State']

In [ ]:
top_k = 10

diffs = X[Y == "H"].mean(axis=0) - X[Y == 'D'].mean(axis=0)
top = np.abs(diffs).sort_values(ascending=False)[0:top_k]
genes = top.index

In [ ]:
X_selected = X[genes]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, Y == "D", test_size=0.3)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)


In [ ]:
y_train_pred = model.predict_proba(X_train)[:, 1]
train_rocauc = roc_auc_score(y_score=y_train_pred, y_true=y_train)
train_prauc = average_precision_score(y_score=y_train_pred, y_true=y_train)
train_accuracy = accuracy_score(y_pred=y_train_pred > 0.5, y_true=y_train)
print("Train quality:")
print(f"ROCAUC : {train_rocauc:.02f}")
print(f"PRAUC : {train_prauc:.02f}")
print(f"Accuracy: accuracy {train_accuracy:.02f}")

In [ ]:
y_test_pred = model.predict_proba(X_test)[:, 1]
train_rocauc = roc_auc_score(y_score=y_test_pred, y_true=y_test)
train_prauc = average_precision_score(y_score=y_test_pred, y_true=y_test)
train_accuracy = accuracy_score(y_pred=y_test_pred > 0.5, y_true=y_test)
print("Test quality:")
print(f"ROCAUC : {train_rocauc:.02f}")
print(f"PRAUC : {train_prauc:.02f}")
print(f"Accuracy: accuracy {train_accuracy:.02f}")

Очевидно, что мы не могли выучить никаких разумных признаков. Однако, мы их все же выучили... Объясните полученный парадокс

Тем не менее, может мы просто что-то делаем не так. Попробуем встроенные в sklearn методы для выбора признаков

In [ ]:
from sklearn.feature_selection import (SelectKBest, SelectFdr, SelectFwe,
                                       SelectPercentile, f_classif)

In [ ]:
X_selected = SelectKBest(f_classif, k=10).fit_transform(X, Y)

Повторите все на отобранных стандартным методом sklearn признаках. Результат объясните

Что делает метод выделения признаков SelectFdr? Что такое FDR? Опишите ниже

Примените метод SelectFdr. Результат объясните

Почему вообще возникла проблема в начале? Объясните

Как бы эту проблему можно было бы избежать?